# 🏷️ Part 2.3 - Extract job skills using LLMs

**Author:** Yu Kyung Koh  
**Last Updated:** July 13, 2025  

---

### 🎯 Objective

* Extract job skills from job postings using LLM
* Specifically, I use **Mistral** model via Ollama, which is free to use and fairly high-performing among the free versions. 
* To ensure consistency across extracted skill terms (e.g., "Microsoft Office" vs. "Microsoft Office Suite"), I apply a **harmonization procedure** that clusters semantically similar skills using **sentence embeddings** and **unsupervised clustering.**
  
### 🗂️ Outline
* **Section 1:** Bring in the job posting data
* **Section 2:** Extract skills using the Mistral model via Ollama
* **Section 3:** Harmonize similar skill terms using embedding + clustering
* **Section 4:** Visualize extracted skills

---
## SECTION 1: Bring in the job posting data 

In [ ]:
import pandas as pd
import os
import re
import joblib
from tqdm import tqdm
from joblib import Parallel, delayed
import math

import nltk
from nltk.corpus import stopwords
#from rapidfuzz import process, fuzz

In [ ]:
# --------------------------------------
# STEP 1: Import data
# --------------------------------------
datadir = '../data/'
jobposting_file = os.path.join(datadir, 'synthetic_job_postings_combined.csv')

posting_df = pd.read_csv(jobposting_file)

In [ ]:
posting_df.head()

In [ ]:
# Check how many job postings are in this data 
len(posting_df)

---
## SECTION 2: Extract skills using the Mistral model via Ollama

* Before running below, we need to type `ollama run mistral` in the terminal
* 

In [ ]:
# --------------------------------------
# STEP 1: Extract skills using the Mistral model
# --------------------------------------
from ollama import chat

# Limit to the first 200 job postings
sample_posting_df = posting_df.head(200).copy()

### Initialize list for storing results
extracted_skills_mistral = []

### Loop through job postings in existing results_df
for desc in tqdm(sample_posting_df["posting_text"]):
    prompt = f"""Extract all job **skills** required in the following job posting.
            Return them as a comma-separated list of keywords only (e.g., Python, Excel, Project Management).
            Include both technical and soft skills.
            Do not include:
                - Job titles (e.g., Educational Program Coordinator)
                - DEI-related terms (e.g., Diversity, Inclusion)
                - Qualifiers like "proficiency", "ability", "skills", or "experience with"
                - Descriptions or explanations — only the canonical skill names

            Job posting:
            \"\"\"{desc}\"\"\"
            """
    response = chat(model='mistral', messages=[
        {'role': 'user', 'content': prompt}
    ])
    
    extracted = response['message']['content']
    extracted_skills_mistral.append(extracted)

### Add new column to results_df
sample_posting_df["extracted_skills_mistral"] = extracted_skills_mistral

In [ ]:
# --------------------------------------
# STEP 2: Examine extracted skills
# --------------------------------------

### 🔷 Comments

* Initial results suggest that the LLM is reasonably effective at extracting job skills from postings.
* However, there are two important caveats:

  1. **Performance and Scalability**
     * Extraction is time-consuming — processing 100 job postings took over 10 minutes.
     * Scaling this to millions of postings may be infeasible with the current setup.
     * A practical alternative for large datasets is to **combine LLMs with machine learning**:
       - Use the LLM to label skill phrases on a small subset of job postings.
       - Train a supervised skill extraction model using these labeled examples.

  2. **Inconsistent Skill Terminology**
     * The same skill can appear under different names across postings (e.g., *Microsoft Office* vs. *Microsoft Office Suite*).
     * To address this, I apply **skill harmonization using embeddings and clustering**.


---
## Section 3: Harmonize similar skill terms using embedding + clustering

In [ ]:
# --------------------------------------
# STEP 1: Parse extracted_skills_mistral into a flat skill list 
# --------------------------------------
import pandas as pd

# Safely split and normalize the extracted skills
sample_posting_df["parsed_skills"] = sample_posting_df["extracted_skills_mistral"].apply(
    lambda x: [s.strip().lower() for s in x.split(",")] if isinstance(x, str) else []
)

In [ ]:
# --------------------------------------
# STEP 2: Embed all skills
# --------------------------------------
from sentence_transformers import SentenceTransformer, util
from sklearn.cluster import AgglomerativeClustering
import numpy as np
from collections import Counter

model = SentenceTransformer("all-MiniLM-L6-v2")

# Flatten and lowercase all skills before embedding
all_skills = sorted(set(skill.strip().lower() for skills in sample_posting_df["parsed_skills"] for skill in skills))

# Get embeddings
embeddings = model.encode(all_skills, convert_to_tensor=True)


In [ ]:
# --------------------------------------
# STEP 3: Cluster skills using Agglomerative clustering
# --------------------------------------

# Cluster similar skills
clustering = AgglomerativeClustering(
    n_clusters=None,
    distance_threshold=0.3,  # try between 0.2–0.4
    linkage='average',
    metric='cosine'
)
labels = clustering.fit_predict(embeddings)


# Create mapping: label → canonical skill (e.g., the shortest skill in group)
cluster_map = {}
for label in set(labels):
    cluster_skills = [s for s, l in zip(all_skills, labels) if l == label]
    if not cluster_skills:
        continue  # skip empty clusters
    canonical = Counter(cluster_skills).most_common(1)[0][0]  # most frequent
    for s in cluster_skills:
        cluster_map[s] = canonical

In [ ]:
# --------------------------------------
# STEP 4: Replace original parsed skills with harmonized version
# --------------------------------------
def harmonize_skills(skill_list):
    return list(set(cluster_map.get(s, s) for s in skill_list))

sample_posting_df["harmonized_skills"] = sample_posting_df["parsed_skills"].apply(harmonize_skills)

---
## Section 4: Visualize extracted skills

In [ ]:
# --------------------------------------
# STEP 1: Combine skills by sector
# --------------------------------------
from collections import defaultdict

# Create a dictionary to hold all skills per sector
sector_skills = defaultdict(list)

for _, row in sample_posting_df.iterrows():
    sector = row["sector"]
    skills = row["harmonized_skills"]
    if isinstance(skills, list):  # skip NaNs or non-lists
        sector_skills[sector].extend(skills)

In [ ]:
# --------------------------------------
# STEP 2: Generate WordClouds per sector
# --------------------------------------
from wordcloud import WordCloud
import matplotlib.pyplot as plt

for sector, skills in sector_skills.items():
    text = " ".join(skills)

    wordcloud = WordCloud(width=800, height=400, background_color="white").generate(text)

    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.title(f"Most common skills (LLM-extracted): Sector {sector} ", fontsize=14)
    plt.tight_layout()
    plt.show()
